# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [2]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [3]:
req = requests.get(url)
req

<Response [200]>

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [4]:
from bs4 import BeautifulSoup

In [5]:
contenido = req.content
sopa = BeautifulSoup(contenido, 'html.parser')
#sopa

In [6]:
sopa_links =sopa.find_all('a')

In [8]:
#sopa_links

In [9]:
links = [i.attrs['href'] for i in sopa.find_all('a') if 'href' in i.attrs ]
   
links [10:20]  

['/wiki/Association_rule_learning',
 '/wiki/Reinforcement_learning',
 '/wiki/Structured_prediction',
 '/wiki/Feature_engineering',
 '/wiki/Feature_learning',
 '/wiki/Online_machine_learning',
 '/wiki/Semi-supervised_learning',
 '/wiki/Unsupervised_learning',
 '/wiki/Learning_to_rank',
 '/wiki/Grammar_induction']

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [10]:
domain = 'http://wikipedia.org'

In [11]:
links_abs = [i for i in links if 'http'in i]
links_rel = [i for i in links if 'http' not in i]

In [12]:
links_rel = [i for i in links_rel if '/' in i]

In [13]:
links_rel[0:2]

['/wiki/Information_science', '/wiki/Machine_learning']

In [14]:
links_abs_c = [i for i in links_abs if '%' not in i]
links_rel_c = [i for i in links_rel if '%' not in i]

In [15]:
links_abs_c[0:3]

['https://arxiv.org/list/cs.LG/recent',
 'https://en.wikipedia.org/w/index.php?title=Template:Machine_learning_bar&action=edit',
 'https://en.wikipedia.org/w/index.php?title=Data_science&action=edit']

In [16]:
links_rel_c[0:3]

['/wiki/Information_science', '/wiki/Machine_learning', '/wiki/Data_mining']

In [17]:
links_relative = [domain + i for i in links_rel_c]

In [18]:
links_relative[0:5]

['http://wikipedia.org/wiki/Information_science',
 'http://wikipedia.org/wiki/Machine_learning',
 'http://wikipedia.org/wiki/Data_mining',
 'http://wikipedia.org/wiki/Statistical_classification',
 'http://wikipedia.org/wiki/Cluster_analysis']

In [19]:
links_total = links_relative + links_abs_c

In [21]:
links_total[0:7]

['http://wikipedia.org/wiki/Information_science',
 'http://wikipedia.org/wiki/Machine_learning',
 'http://wikipedia.org/wiki/Data_mining',
 'http://wikipedia.org/wiki/Statistical_classification',
 'http://wikipedia.org/wiki/Cluster_analysis',
 'http://wikipedia.org/wiki/Regression_analysis',
 'http://wikipedia.org/wiki/Anomaly_detection']

In [22]:
len(links_total)

353

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [23]:
import os

In [24]:
os.getcwd()

'/Users/macbookair13/Desktop/ironhack/datamex_102020/module-1/lab-parallelization/your-code'

In [26]:
#os.mkdir('wikipedia')

In [27]:
os.listdir()

['main.ipynb', 'Learning.ipynb', 'wikipedia', '.ipynb_checkpoints']

In [28]:
os.chdir("wikipedia")

In [29]:
os.getcwd()

'/Users/macbookair13/Desktop/ironhack/datamex_102020/module-1/lab-parallelization/your-code/wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [55]:
pip install python-slugify

     |████████████████████████████████| 78 kB 750 kB/s eta 0:00:01
  Created wheel for python-slugify: filename=python_slugify-4.0.1-py2.py3-none-any.whl size=6767 sha256=f530797d76a7fb5a48acc5dcf21c8d9afddec8c44a0816c1656c5a97180249f8
  Stored in directory: /Users/macbookair13/Library/Caches/pip/wheels/48/1b/6f/5c1cfab22eacbe0095fc619786da6571b55253653c71324b5c
Successfully built python-slugify
Note: you may need to restart the kernel to use updated packages.


In [30]:
from slugify import slugify

In [71]:
def index_page(url):
    try:
        req = requests.get(url)
        html = slugify(url) + '.html'
        contenido = req.content
        sopa = BeautifulSoup(contenido, 'html.parser').text
        archivo = open(html,'w',encoding='utf-8')
        archivo.write(sopa)
        archivo.close()
    except:
        pass
    return archivo

In [73]:
#prueba:index_page('http://wikipedia.org/wiki/Machine_learning')

<_io.TextIOWrapper name='http-wikipedia-org-wiki-machine-learning.html' mode='w' encoding='utf-8'>

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [66]:
%%time
index_page('http://wikipedia.org/wiki/Machine_learning')

CPU times: user 441 ms, sys: 20.9 ms, total: 462 ms
Wall time: 1.13 s


<_io.TextIOWrapper name='http-wikipedia-org-wiki-machine-learning.html' mode='w' encoding='utf-8'>

In [67]:
links_total_reduce = links_total[0:10]
#reducimos la lista

In [55]:
os.getcwd()

'/Users/macbookair13/Desktop/ironhack/datamex_102020/module-1/lab-parallelization/your-code/wikipedia'

In [80]:
%%time
for i in links_total_reduce:
    index_page(i)

CPU times: user 3.29 s, sys: 172 ms, total: 3.46 s
Wall time: 9.52 s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [69]:
import multiprocessing as mp

In [78]:
%%time
pool = mp.Pool(mp.cpu_count())
res = pool.map(index_page,links_total_reduce)
pool.close()
